### Libraries

In [1]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils   
import datetime        

'machine learning imports'
import torch
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.pretraining import TabNetPretrainer
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### GPU

In [2]:
DEVICE = 'CUDA' if torch.cuda.is_available() else 'CPU'
print("Using {}".format(DEVICE))

# Info on the device available memory
if DEVICE == 'CUDA':
    
    gpu = torch.device('cuda')
    total_memory = torch.cuda.get_device_properties(gpu).total_memory / 1024**3
    current_memory = torch.cuda.memory_allocated(gpu) / 1024**3

    print(f'Total GPU memory: {total_memory:.1f} GB | Current usage: {current_memory:.1f} GB')

Using CPU


### Load data

In [3]:
# Non-augmented dataset
df_train = pd.read_csv('data/train_smotenc.csv', low_memory=False)
AUGMENTATION = 'None'

# SMOTE augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE'

# SMOTE-NC augmented dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
# AUGMENTATION = 'SMOTE-NC'

# RealTabFormer augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
# AUGMENTATION = 'RealTabFormer'

# GReaT augmentation dataset
# df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
# AUGMENTATION = 'GReaT'


# Test data for all datasets
df_test = pd.read_csv('data/EdgeIIot_test.csv', low_memory=False)

### Data preparation

In [4]:
# drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
#df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
#df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

# Extract categorical features
categorical_features = X_train.select_dtypes(include="object").columns

# Extract indices of categorical features
cat_idxs = [X_train.columns.get_loc(col) for col in categorical_features]

# Find number of unique values in each categorical column
cat_dims = [len(X_train[col].unique()) for col in categorical_features]

#### One-hot encoding categorical features

In [5]:
# Concatenate X_train and X_test to get all possible values for categorical columns
X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# Apply one-hot encoding (get_dummies)
X_comb_enc = pd.get_dummies(X_comb, columns=categorical_features)

# Split X_comb_enc into X_train_enc and X_test_enc
X_train_enc = X_comb_enc.iloc[:X_train.shape[0], :]
X_test_enc = X_comb_enc.iloc[X_train.shape[0]:, :]

# # Drop categorical columns from X_train and X_test
# X_train = X_train.drop(categorical_features, axis=1)
# X_test = X_test.drop(categorical_features, axis=1)

# # Concatenate X_train and X_test with X_train_enc and X_test_enc and drop index column
# X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
# X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)

# Print the shape of X_train and X_test
print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (1500000, 53), X_test_enc shape: (381934, 53)


In [6]:
# # AQUI Categorical columns in df_train
# categorical_columns = [f for f in features if f in df_train.select_dtypes(include="object").columns]

# # Concatenate X_train and X_test
# X_comb = pd.concat([X_train[categorical_columns], X_test[categorical_columns]], axis=0)

# # Apply one-hot encoding (get_dummies)
# X_comb_enc = pd.get_dummies(X_comb, dtype='int8')

# # Split back into X_train and X_test
# X_train_enc, X_test_enc = train_test_split(
#     X_comb_enc, test_size=len(X_test), random_state=42)

# # Print the shape of X_train_enc and X_test_enc
# print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')


# # converts X_train and y_train to numpy arrays
# X_train = df_train[features]
# y_train = df_train["Attack_type"]

# # converts X_test and y_test to numpy arrays
# X_test = df_test[features]
# y_test = df_test["Attack_type"]

# # size of X_train, y_train  X_test, y_test
# print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
# print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
# print(cat_idxs)

#### Label encoding

In [7]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model Training

In [8]:
# # embedding dimension for each categorical column
# cat_emb_dim = 10 

# # initialize embedder 
# cat_embedder = TabNetPretrainer(cat_dims, cat_emb_dim, cat_idxs)

# # instantiate TabNetClassifier model
# tabnet = TabNetClassifier(device_name = DEVICE,
#                           cat_idxs=cat_idxs,
#                           cat_dims=cat_dims,
#                           cat_emb_dim=cat_emb_dim)

In [9]:
# tabnet = TabNetClassifier(n_d=64, 
#                           n_a=64, 
#                           n_steps=5,
#                           gamma=1.5,
#                           cat_idxs=cat_idxs,
#                           cat_dims=cat_dims,
#                           cat_emb_dim=10,
#                           lambda_sparse=1e-4, 
#                           momentum=0.3, 
#                           clip_value=2.,
#                           optimizer_params=dict(lr=2e-2),
#                           scheduler_params = {"gamma": 0.95, "step_size": 20},
#                           scheduler_fn=torch.optim.lr_scheduler.StepLR, 
#                           )

# pytorch_tabnet default parameters default parameters except for cat_emb_dim, which is set to 10
tabnet = TabNetClassifier(cat_idxs=cat_idxs,
                          cat_dims=cat_dims,
                          cat_emb_dim=10,
                          )

C:\Users\rogerio.l.costa\Anaconda3\envs\py311\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [10]:
# TabModel(n_d: int = 8,
#         n_a: int = 8, 
#         n_steps: int = 3, 
#         gamma: float = 1.3, 
#         cat_idxs: List[int] = <factory>, 
#         cat_dims: List[int] = <factory>, 
#         cat_emb_dim: int = 1, 
#         n_independent: int = 2, 
#         n_shared: int = 2, 
#         epsilon: float = 1e-15, 
#         momentum: float = 0.02, 
#         lambda_sparse: float = 0.001, 
#         seed: int = 0, 
#         clip_value: int = 1, 
#         verbose: int = 1, 
#         optimizer_fn: Any = <class 'torch.optim.adam.Adam'>, 
#         optimizer_params: Dict = <factory>, 
#         scheduler_fn: Any = None, 
#         scheduler_params: Dict = <factory>, 
#         mask_type: str = 'sparsemax', 
#         input_dim: int = None, 
#         output_dim: int = None, 
#         device_name: str = 'auto', 
#         n_shared_decoder: int = 1, 
#         n_indep_decoder: int = 1, 
#         grouped_features: List[List[int]] = <factory>
#         )

# fit(X_train, 
#     y_train, 
#     eval_set=None, 
#     eval_name=None, 
#     eval_metric=None, 
#     loss_fn=None, 
#     weights=0, 
#     max_epochs=100, 
#     patience=10, 
#     batch_size=1024, 
#     virtual_batch_size=128, 
#     num_workers=0, 
#     drop_last=True, 
#     callbacks=None, 
#     pin_memory=True, 
#     from_unsupervised=None, 
#     warm_start=False, 
#     augmentations=None, 
#     compute_importance=True)

In [ ]:
# pytorch_tabnet default parameters
tabnet.fit(X_train=X_train_enc.values, 
           y_train=y_train,
           augmentations=None,
           )

C:\Users\rogerio.l.costa\Anaconda3\envs\py311\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 1.36191 |  0:03:40s
epoch 1  | loss: 1.31597 |  0:07:58s
epoch 2  | loss: 1.3159  |  0:10:13s
epoch 3  | loss: 1.31543 |  0:12:41s
epoch 4  | loss: 1.31699 |  0:15:13s
epoch 5  | loss: 1.31468 |  0:17:49s
epoch 6  | loss: 1.31455 |  0:20:24s
epoch 7  | loss: 1.31657 |  0:22:52s
epoch 8  | loss: 1.31837 |  0:25:21s
epoch 9  | loss: 1.31664 |  0:27:46s
epoch 10 | loss: 1.31654 |  0:30:27s
epoch 11 | loss: 1.31647 |  1:20:36s
epoch 12 | loss: 1.31639 |  1:22:57s
epoch 13 | loss: 1.31632 |  1:25:18s
epoch 15 | loss: 1.31634 |  1:30:42s
epoch 16 | loss: 1.31625 |  1:33:07s
epoch 17 | loss: 1.31687 |  1:35:25s
epoch 18 | loss: 1.31624 |  1:37:44s
epoch 19 | loss: 1.3161  |  1:39:27s
epoch 20 | loss: 1.31598 |  1:40:47s
epoch 21 | loss: 1.31597 |  1:42:07s
epoch 22 | loss: 1.31598 |  1:43:26s
epoch 23 | loss: 1.31634 |  1:44:46s
epoch 24 | loss: 1.31613 |  1:46:05s
epoch 25 | loss: 1.31612 |  1:47:28s
epoch 26 | loss: 1.31602 |  1:48:53s
epoch 27 | loss: 1.31596 |  1:50:17s
e

In [ ]:
# NOTA: NÃO É nËCESSÄRIO APAGAR COLUNAS CATEGÓRICAS DE X_TRAIN E X_TEST SE SE IDENTIFICAREM OS ÍNDICES DAS COLUNAS CATEGÓRICAS E SE SE UTILIZAR O PARÂMETRO cat_idxs=cat_idxs NO MODELO TABNETCLASSIFIER
# REVER OUTROS NOTEBOOKS!!!

#### Save model

In [ ]:
saved_filename = tabnet.save_model('checkpoints/tabnet_none')

### Model Evaluation

In [14]:
predictions = tabnet.predict(X_test_enc.values)

#### Metrics

In [15]:
accuracy = metrics.accuracy_score(y_test, predictions)
precisionw = metrics.precision_score(y_test, predictions, average='weighted', zero_division=1)
recallw = metrics.recall_score(y_test, predictions, average='weighted')
f1_scorew = metrics.f1_score(y_test, predictions, average='weighted')
precisionm = metrics.precision_score(y_test, predictions, average='macro', zero_division=1)
recallm = metrics.recall_score(y_test, predictions, average='macro')
f1_scorem = metrics.f1_score(y_test, predictions, average='macro')


print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Precision (Weighted): {:.2f}".format(precisionw))
print("Recall (Weighted): {:.2f}".format(recallw))
print("F1(Weighted): {:.2f}".format(f1_scorew))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Precision (Macro): {:.2f}".format(precisionm))
print("Recall (Macro): {:.2f}".format(recallm))
print("F1(Macro): {:.2f}".format(f1_scorem))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.78
~~~~~~~~~~~~~~~~~~~~~~~~~
Precision (Weighted): 0.96
Recall (Weighted): 0.78
F1(Weighted): 0.76
~~~~~~~~~~~~~~~~~~~~~~~~~
Precision (Macro): 0.79
Recall (Macro): 0.36
F1(Macro): 0.26
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [16]:
# create dictionary for results
results = {
    "model": "tabnet",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision (weighted)": precisionw,
    "recall (weighted)": recallw,
    "f1 (weighted)": f1_scorew,
    "precision (macro)": precisionm,
    "recall (macro)": recallm,
    "f1 (macro)": f1_scorem
    }
# save results to csv   
utils.save_results_to_csv([results], 'results/metrics/tabnet.csv')

#### Confusion matrix

In [17]:
conf_mat = metrics.confusion_matrix(y_test, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,0,0,0,0,0,0,0,0,0,0,4782,0,0,0,0
DDoS_HTTP,0,9628,0,0,0,0,0,0,0,0,0,0,0,0,0
DDoS_ICMP,0,0,0,0,0,0,0,0,0,0,13501,0,0,0,0
DDoS_TCP,0,0,0,0,0,0,0,0,0,0,10009,0,0,0,0
DDoS_UDP,0,0,0,0,0,0,0,0,0,0,24601,0,0,0,0
Fingerprinting,0,0,0,0,0,0,0,0,0,0,146,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,2,272773,0,0,1,0,0,0,0
Password,0,0,0,0,0,0,0,0,10108,0,0,0,0,0,0
